# Первоначальная настройка

Скачивание необходимых библиотек

In [ ]:
!pip install transformers sentencepiece

In [ ]:
!pip install torch

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!rm -r sample_data

Импорт бибилиотек

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
import os
from datasets import load_dataset
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset
import random
import gc

# Datasets


In [ ]:
def task():
    return input_text, output_text

### Создание задачи(quiz)

In [ ]:
!gdown --id 13n_UBlkCyvy28GDa-tTfR2KCtaIHyjBB && mkdir dataset && cp quiz.tsv /content/dataset/quiz.tsv

zsh:1: command not found: gdown


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/quiz.tsv', sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/datasets/quiz.tsv'

In [ ]:
df_train, df_test = train_test_split(df.dropna(), test_size=0.5, random_state=1)
pairs = df_train[['q', 'a']].values.tolist()

### Создание задачи xlsum(sum)

In [ ]:
xlsum = load_dataset("csebuetnlp/xlsum", "russian", cache_dir="/content/dataset")

OSError: [Errno 30] Read-only file system: '/content'

In [ ]:
print(random.choice(xlsum['train']))

In [ ]:
def xlsum_task():
    item = random.choice(xlsum['train'])
    if random.random() < 0.5:
      return f'sum_short | {item["text"]}', item["title"]
    else:
      return f'sum_long | {item["text"]}', item["summary"]
xlsum_task()

### Создание задачи IlyaGusev/gazeta(sum)

In [ ]:
gazeta = load_dataset('IlyaGusev/gazeta', revision="v2.0", cache_dir="/content/dataset")

In [ ]:
print(random.choice(gazeta['train']["text"]))

In [ ]:
def gazeta_task():
    item = random.choice(gazeta['train'])
    if random.random() < 0.5:
      return f'sum_short | {item["text"]}', item["title"]
    else:
      return f'sum_long | {item["text"]}', item["summary"]
gazeta_task()

### Создание задачи opus_wiki(translate)

In [ ]:
opus_wiki = load_dataset("Helsinki-NLP/opus_wikipedia", "en-ru", cache_dir="/content/dataset")

In [ ]:
random.choice(opus_wiki['train'])

In [ ]:
def translate_task():
    item = random.choice(opus_wiki['train'])['translation']
    if random.random() < 0.5:
        return f'translate ru-en | {item["ru"]}', item["en"]
    else:
        return f'translate en-ru | {item["en"]}', item["ru"]

translate_task()

### Создание задачи tapaco(paraphrase)

In [ ]:
tapaco  = load_dataset('tapaco', 'ru', cache_dir="/content/dataset")

In [ ]:
from collections import Counter, defaultdict
from tqdm.auto import tqdm, trange
cnt = Counter()
p2s = defaultdict(list)
for i, e in enumerate(tqdm(tapaco['train'])):
    cnt[e['paraphrase_set_id']] += 1
    p2s[e['paraphrase_set_id']].append(i)

In [ ]:
ph = random.choice(tapaco['train'])['paraphrase_set_id']

In [ ]:
def paraphrase_task():
    ph = random.choice(tapaco['train'])['paraphrase_set_id']
    texts = tapaco['train'][p2s[ph]]['paraphrase']
    random.shuffle(texts)
    return f'paraphrase | {texts[0]}', texts[1]

paraphrase_task()

### Вопрос-ответ

In [ ]:
!wget https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Conversations/Data/ru.conversations.txt

In [ ]:
with open('ru.conversations.txt', 'r') as f:
    blobs = f.read().split('\n\n')
print(len(blobs))

In [ ]:
def reply_task():
    b = random.choice(blobs)
    phrases = b[2:].split('\n- ')
    if len(phrases) < 2:
        return reply_task()
    split_point = random.randint(1, len(phrases)-1)
    prefix = '\n\n'.join(phrases[:split_point])
    return f'reply | {prefix}', phrases[split_point]

reply_task()

### Question answering

In [ ]:
!wget http://files.deeppavlov.ai/datasets/sber_squad-v1.1.tar.gz

In [ ]:
!tar -xvzf sber_squad-v1.1.tar.gz

In [ ]:
import json
with open('train-v1.1.json', 'r') as f:
    sbsq = json.load(f)

In [ ]:
len(sbsq['data'][0]['paragraphs'])

In [ ]:
def ask_task():
    pp = random.choice(sbsq['data'][0]['paragraphs'])
    qq = random.choice(pp['qas'])
    return f'ask | {pp["context"]}', qq["question"]

ask_task()

In [ ]:
def comprehend_task():
    pp = random.choice(sbsq['data'][0]['paragraphs'])
    qq = random.choice(pp['qas'])
    aa = random.choice(qq['answers'])
    return f'comprehend | {pp["context"]}.\nВопрос: {qq["question"]}', aa["text"]

comprehend_task()

### Russian SuperGLUE

In [ ]:
!for TASK in LiDiRus RCB PARus MuSeRC TERRa RUSSE RWSD DaNetQA RuCoS; do wget https://russiansuperglue.com/tasks/download/$TASK --content-disposition && unzip $TASK.zip; done
!rm -rf ./*.zip

In [ ]:
import codecs
import json
import pandas as pd
from collections import defaultdict
import copy
import random
import numpy as np
import re

def load_jsonl(path):
    with codecs.open(path, encoding='utf-8-sig') as reader:
        lines = reader.read().split("\n")
        lines = list(map(json.loads, filter(None, lines)))
    return lines

In [ ]:
TASK_NAMES = ['LiDiRus', 'RCB', 'PARus', 'MuSeRC', 'TERRa', 'RUSSE', 'RWSD', 'DaNetQA', 'RuCoS']

task_data = {k: load_jsonl(f'{k}/train.jsonl') for k in TASK_NAMES[1:]}

task_ids = list(task_data.keys())
task_weights = np.array([len(task_data[t]) for t in task_ids])**0.5
task_weights /= task_weights.sum()
print(task_ids)
print(task_weights)

In [ ]:
RCB_labels = {
    'contradiction': 'противоречие',
    'entailment': 'следствие',
    'neutral': 'не очевидно'
}

def RCB_sampler(item):
    return f'Дано: {item["premise"]}. Гипотеза: {item["hypothesis"]}. Логично?', RCB_labels[item["label"]]

def PARus_sampler(item):
    if item['question'] == 'cause':
        q, a = 'Следствие', 'Причина'
    else: # effect
        q, a = 'Причина', 'Следствие'
    idx = random.choice([0, 1])
    ch = item['choice' + str(idx + 1)]
    ans = 'да' if item['label'] == idx else 'нет'
    parts = [f'{q}: {item["premise"]}', f"{a}: {ch}"]
    random.shuffle(parts)
    return f'{parts[0]}. {parts[1]}. Логично?', ans

def MuSeRC_sampler(item):
    text = re.sub('\(\d+\) ', '', item['passage']['text'])
    qq = random.choice(item['passage']['questions'])
    q = qq['question']
    a = random.choice([a for a in qq['answers'] if a['label']])['text']
    return f'Вопрос: {q}. Дано: {text}', a

TERRa_labels = {
    'entailment': 'логично',
    'not_entailment': 'не очевидно'
}

def TERRa_sampler(item):
    return f'Дано: {item["premise"]}. Гипотеза: {item["hypothesis"]}. Логично?', TERRa_labels[item["label"]]

def RUSSE_sampler(item):
    sents = [item['sentence1'], item['sentence2']]
    random.shuffle(sents)
    ans = 'да' if item['label'] else 'нет'
    return f'Слово "{item["word"]}" употребляется в одинаковом смысле? Текст 1: {sents[0]}. Текст 2: {sents[1]}.', ans

def RWSD_sampler(item):
    s1, s2 = item['target']['span1_text'], item['target']['span2_text']
    if item['label'] and random.random() < 0.5:
        return f'{item["text"]}. К чему относится "{s2}"?', s1
    return f'{item["text"]}. "{s2}" относится к "{s1}"?', 'да' if item['label'] else 'нет'

def DaNetQA_sampler(item):
    text = item["passage"]
    q = item["question"].replace('?', '')
    ans = 'да' if item['label'] else 'нет'
    if random.random() < 0.5:
        return f'{text}. Вопрос: {q}?', ans
    else:
        return f'Вопрос: {q}? {text}', ans

def RuCoS_sampler(item):
    text = item["passage"]['text'].split('@highlight')[0].strip()
    qa = random.choice(item['qas'])
    q, a = qa['query'], random.choice(qa['answers'])['text']
    if random.random() < 0.5:  #summarization
        return text + '\nВкратце:', q.replace('@placeholder', a)
    else:
        summary = q.replace('@placeholder', 'ЭТО')
        return f'{summary}. {text}. Вопрос: ЭТО - что?', a

samplers = {
    'RCB': RCB_sampler,
    'PARus': PARus_sampler,
    'MuSeRC': MuSeRC_sampler,
    'TERRa': TERRa_sampler,
    'RUSSE': RUSSE_sampler,
    'RWSD': RWSD_sampler,
    'DaNetQA': DaNetQA_sampler,
    'RuCoS': RuCoS_sampler,
}

def rsg_task(task=None):
    if task is None:
        task = random.choices(task_ids, weights=task_weights)[0]
    q, a = samplers[task](random.choice(task_data[task]))
    return f'{task} | {q}', a

In [ ]:
for t in task_ids:
    q, a = rsg_task(task=t)
    print(q)
    print(a)
    print()

### A special Russian text normalization task

In [ ]:
!pip install natasha

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)


def text2doc(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return doc

In [ ]:
!pip install pymorphy2

In [ ]:
from pymorphy2 import MorphAnalyzer
anl = MorphAnalyzer()

from natasha.morph.vocab import OC_UD_INDEX, OC_UD_FEATS, OC_UD_POS
from natasha.norm import normal_pos
fvalue_pm_dict = {}
nat2pm = {}

for fname, fvalue_pm, fvalue_nat in OC_UD_FEATS:
    fvalue_pm_dict[fvalue_pm.lower()] = (fname, fvalue_nat)
    nat2pm[(fname, fvalue_nat)] = fvalue_pm.lower()

In [ ]:
from collections import Counter, defaultdict

feature_counter = defaultdict(Counter)
pos_counter = Counter()

IMPORTANT_POS = {'NOUN', 'PROPN', 'VERB', 'ADJ', 'DET', 'PRON', 'PART', 'ADV', 'NUM', 'X', 'INTJ'}
HELPER_POS = {'ADP', 'PROPN', 'PUNCT', 'SCONJ', 'CCONJ', 'AUX', 'SYM'}

for i, item in enumerate(tqdm(tapaco['train'])):
    if i > 10000:
        break
    doc = text2doc(item['paraphrase'])
    for token in doc.tokens:
        for k, v in token.feats.items():
            feature_counter[k][v] += 1
        pos_counter[token.pos] += 1

        if token.pos not in IMPORTANT_POS.union(HELPER_POS):
            print(token.text)
            print(token.pos)
            print()

In [ ]:
def find_best_parse(token, parses):
    feats = {(k, v) for k, v in token.feats.items()}
    #print(feats)
    scores = []
    for p in parses:
        cand_feats = set()
        score = 1
        for g in p.tag.grammemes:
            if g in OC_UD_POS:
                if str(g) == normal_pos(token.pos) or OC_UD_POS[g] == normal_pos(token.pos):
                    score -= 1
            elif g in fvalue_pm_dict:
                cand_feats.add(fvalue_pm_dict[g])
            elif g in {'Sgtm', 'Geox', 'Name', 'intr', 'tran', 'intg', 'UNKN'}:
                continue
        score = len(feats.difference(cand_feats)) + len(cand_feats.difference(feats)) - p.score
        scores.append(score)
        #print(cand_feats, score)
    return parses[np.argmin(scores)]

In [ ]:
def reinflect(token, p_lemma=0.5) -> str:
    """ Try to reinflect a token into something random """
    if token.pos == 'PUNCT':
        return token.text
    if token.lemma and random.random() < p_lemma:
        return token.lemma
    parses = anl.parse(token.text)
    bp = find_best_parse(token, parses)
    if not bp:
        return token.lemma or token.text
    new_feats = []
    for k, v in token.feats.items():
        if k in {'Animacy', 'Aspect'}:
            continue
        if bp.tag.POS in {'NOUN'} and k in {'Gender'}:
            continue
        if 'Sgtm' in bp.tag and k == 'Number':
            continue

        keys = list(feature_counter[k].keys())
        if not keys:
            continue
        new_v_nat = random.choice(keys)
        if (k, new_v_nat) not in nat2pm:
            continue
        new_v_pm = nat2pm[(k, new_v_nat)]

        if bp.tag.aspect and bp.tag.aspect == 'perf' and new_v_pm == 'pres':
            continue

        if (k, new_v_nat) in nat2pm:
            new_feats.append(new_v_pm)
    random.shuffle(new_feats)
    infl = None
    newp = bp
    if new_feats:
        for f in new_feats:
            try:
                infl = newp.inflect({f})
            except ValueError:
                continue
            if infl:
                newp = infl
    if newp.word != bp.word:
        return newp.word
    return bp.word

In [ ]:
def spoil_text(doc, reinflection=0.5, important_drop=0.2, helper_drop=0.5, position_noise=2.0):
    results = []
    for token in doc.tokens:
        if token.pos in IMPORTANT_POS:
            if random.random() < important_drop:
                continue
        else:
            if random.random() < helper_drop:
                continue
        word = token.text
        if random.random() < reinflection:
            word = reinflect(token)
        results.append(word)
    if position_noise:
        orders = [random.normalvariate(mu=i, sigma=position_noise) for i, _ in enumerate(results)]
        results = [x for _, x in sorted(zip(orders, results))]
    if not results:
        return random.choice(doc.tokens).text.lower()
    return ' '.join(results).lower()

spoil_text(text2doc('Шла Саша по шоссе и сосала сушку.'), reinflection=0, important_drop=0, helper_drop=0)

In [ ]:
import numpy as np

def assemble_task():
    text = random.choice(df_leipzig.text)
    spoiled = spoil_text(
        text2doc(text),
        reinflection=random.random() * 1.0,
        important_drop=random.random() * 0.5,
        helper_drop=random.random() * 0.8,
        position_noise=np.exp(random.normalvariate(0, 1)),
    )
    return f'assemble | {spoiled}', text

assemble_task()

### Po delu dataset

In [ ]:
po_delu_dataset = load_dataset("slon-hk/BooksSummarizationRU")
item = random.choice(po_delu_dataset['train'])
print(item)

{'ID': 122, 'Title': 'Скворцы', 'Author': 'Василий Белов', 'Summary': 'Названия глав — условные.\nМальчик Павлуня\nВ субботу мама вымыла пол, вычистила мелким песочком самовар, так что он начал сиять, поставила его в шкаф и передвинула кровать Павлуни ближе к окну, а затем ушла на работу в колхоз.\n👩🏻\nМама — забот\xadли\xadвая и тру\xadдо\xadлю\xadби\xadвая жен\xadщина, все\xadгда заня\xadтая рабо\xadтой, раз\xadго\xadва\xadри\xadвает с сыном лас\xadково, неж\xadная и вни\xadма\xadтель\xadная.\n👦🏻\nПавлуня — боль\xadной маль\xadчик, не может ходить, обла\xadдает бога\xadтой фан\xadта\xadзией и чув\xadстви\xadтель\xadным серд\xadцем, неж\xadный, вос\xadпри\xadим\xadчи\xadвый, любо\xadзна\xadтель\xadный.\nПавлуня хотел посмотреть, освещает ли самовар шкаф изнутри или нет. Ещё Павлуне хотелось рассмотреть новые валенки, которые отец купил ему на зиму, но они лежали в чулане. Ни до шкафа, ни до чулана мальчик добраться не мог. Всю зиму у него «болели ноги, и он лежал всё время в кровати»,

In [ ]:
def po_delu_dataset():
  return f'sum | {item["Full Text"]}', item["Summary"]

print(po_delu_dataset())

('sum | \n\n\n\n\n\n\n\n\n\n\n\n\nСкворцы\n\n\n\n\n\n\n\n\n\n\n\nАлександр Куприн\n\nСкворцы\n\n\nБыла середина марта. Весна в этом году выдалась ровная, дружная. Изредка выпадали обильные, но короткие дожди. Уже ездили на колесах по дорогам, покрытым густой грязью. Снег еще лежал сугробами в глубоких лесах и в тенистых оврагах, но на полях осел, стал рыхлым и темным, и из-под него кое-где большими плешинами показалась черная, жирная, парившаяся на солнце земля. Березовые почки набухли. Барашки на вербах из белых стали желтыми, пушистыми и огромными. Зацвела ива. Пчелы вылетели из ульев за первым взятком. На лесных полянах робко показались первые подснежники.\nМы с нетерпением ждали, когда к нам в сад опять прилетят старые знакомые – скворцы, эти милые, веселые, общительные птицы, первые перелетные гости, радостные вестники весны. Много сотен верст нужно им лететь со своих зимних становищ, с юга Европы, из Малой Азии, из северных областей Африки. Иным придется сделать побольше трех тыс

# Обучение

### Дообучение на датасетах

In [ ]:
raw_model = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(raw_model)
tokenizer = T5Tokenizer.from_pretrained(raw_model)

config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
device = torch.device('cuda')
model.to(device)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
optimizer = torch.optim.Adam(params = [p for p in model.parameters() if p.requires_grad], lr=1e-5)

In [ ]:
TASKS = [
    xlsum_task,
    gazeta_task,
    translate_task,
    paraphrase_task,
    reply_task,
    ask_task,
    comprehend_task,
    rsg_task,
    po_delu_dataset
]
len(TASKS)

1

In [ ]:
def predict(x, n=3):
    inputs = tokenizer(x, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        hypotheses = model.generate(
            **inputs,
            do_sample=True,
            top_p=0.9,
            num_return_sequences=3,
            repetition_penalty=2.5,
            max_length=64,
        )
    return [tokenizer.decode(h, skip_special_tokens=True) for h in hypotheses]

In [ ]:
model.eval()

for t in TASKS:
    x, y = t()
    print(x, ' \n --> ', y)
    for p in predict(x, n=3):
        print(p)
    print()

sum | 












Скворцы











Александр Куприн

Скворцы


Была середина марта. Весна в этом году выдалась ровная, дружная. Изредка выпадали обильные, но короткие дожди. Уже ездили на колесах по дорогам, покрытым густой грязью. Снег еще лежал сугробами в глубоких лесах и в тенистых оврагах, но на полях осел, стал рыхлым и темным, и из-под него кое-где большими плешинами показалась черная, жирная, парившаяся на солнце земля. Березовые почки набухли. Барашки на вербах из белых стали желтыми, пушистыми и огромными. Зацвела ива. Пчелы вылетели из ульев за первым взятком. На лесных полянах робко показались первые подснежники.
Мы с нетерпением ждали, когда к нам в сад опять прилетят старые знакомые – скворцы, эти милые, веселые, общительные птицы, первые перелетные гости, радостные вестники весны. Много сотен верст нужно им лететь со своих зимних становищ, с юга Европы, из Малой Азии, из северных областей Африки. Иным придется сделать побольше трех тысяч верст. Многие пролетят над мор

In [ ]:
for t in task_ids:
    q, a = rsg_task(task=t)
    print(q)
    print(a)
    for p in predict(q, n=3):
        print(p)
    print()

NameError: name 'task_ids' is not defined

In [ ]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [ ]:
optimizer.param_groups[0]['lr'] = 1e-5

In [ ]:
def eval_losses(n=10, max_len=1024):
    for task in TASKS:
        tot = 0
        for i in range(n):
            xxx, yyy = task()
            x = tokenizer(xxx, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
            y = tokenizer(yyy, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)

            loss = model(
                input_ids=x.input_ids,
                attention_mask=x.attention_mask,
                labels=y.input_ids,
                decoder_attention_mask=y.attention_mask,
                return_dict=True
            ).loss
            loss.backward()
            tot += loss.item()
        print(f'{task.__name__:20s} {tot / n :2.2f}')

eval_losses(n=20)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
model.train();
batch_size = 2
max_len = 1024
epochs = 5
accumulation_steps = 32
save_steps = 5000

window = 5000
ewm = 0

tq = trange(int(100000000 / batch_size))
cleanup()

for i in tq:
    xx = []
    yy = []
    for _ in range(batch_size):
        xxx, yyy = random.choice(TASKS + [rsg_task] * 3)()  # rsg is more various, increase its occurrence 4-fold
        xx.append(xxx)
        yy.append(yyy)

    try:
        x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
        y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_len).to(device)
        # do not force the model to predict pad tokens
        y.input_ids[y.input_ids==0] = -100

        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        # print('ok')
    except RuntimeError:
        print([xxx.split(' |')[0] for xxx in xx])
        loss = None
        cleanup()
        continue

    w = 1 / min(i+1, window)
    ewm = ewm * (1-w) + loss.item() * w
    tq.set_description(f'loss: {ewm}')

    if i % accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
        cleanup()

    if i % window == 0 and i > 0:
        print(ewm)
        cleanup()
        # optimizer.param_groups[0]['lr'] *= 0.999
    if i % save_steps == 0 and i > 0:
        model.save_pretrained(MODEL_NAME)
        tokenizer.save_pretrained(MODEL_NAME)
        print('saving...', i, optimizer.param_groups[0]['lr'])

        try:
            optimizer.step()
            optimizer.zero_grad()
            eval_losses()
            optimizer.step()
            optimizer.zero_grad()
        except RuntimeError:
            cleanup()

In [ ]:
MODEL_NAME = "third_model"
model.save_pretrained(MODEL_NAME)
tokenizer.save_pretrained(MODEL_NAME)

# Пример суммаризации

Разбиваем токены на части и декодируем их обратно в текст

In [10]:
def split_text(text, max_tokens=512):
    tokens = tokenizer(text, return_tensors='pt', truncation=False).input_ids[0]
    return [tokenizer.decode(tokens[i:i + max_tokens], skip_special_tokens=True)
            for i in range(0, len(tokens), max_tokens)]

In [11]:
MODEL_NAME = 'slon-hk/PoDeluModel'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda();
model.eval();

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
def summarize(
    text, n_words=None, compression=None,
    max_length=1000, min_length=None, num_beams=3, do_sample=False, repetition_penalty=10.0,
    **kwargs
):
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x,
            max_length=max_length, num_beams=num_beams,
            do_sample=do_sample, repetition_penalty=repetition_penalty,
            min_length=min_length,
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [1]:
def console_out():
  text = input("Введите текст для сжатия:")
  n = int(input("Введите колличество слов до которого вы хотите сжать текст: "))
  print(summarize(text, n_words=n, min_length=n, num_beams=25, repetition_penalty=2.5, do_sample=True))
console_out()

KeyboardInterrupt: Interrupted by user